In [1]:
import torch
import torchvision
from torch import nn 
from torchvision import transforms
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image 
import numpy as np
from scipy.io import loadmat
from scipy.io import savemat 
import sys
sys.path.append('../')
import dataset_utils

In [2]:
def get_imgs_bold_id(image_dataset,df):
  img2dna = dict()
  not_found_images = []
  for i, row in df.iterrows():
        url = row['image_urls'].split('|')[0]
        genus_name = row['genus_name'].replace(' ','_')
        image_name_csv ='image_dataset/' + genus_name + '/' + url[url.rfind('/')+1:]
        trovato = False
        for img in image_dataset.imgs:
            if img[0] == image_name_csv:
                img2dna[img[0]]= row['processid']
                trovato = True
                break
        if not trovato:
            not_found_images.append(image_name_csv)
  return img2dna

In [3]:
raise Exception
import pandas as pd
import requests
import threading
import time

df = pd.read_csv('unknown_species_new_samples.csv')
df['nucleotide'] = None  
missing_bolds = []
batch_size = 500
start_time = time.time()

def fetch_data(batch_ids, index):
    url = f"http://v3.boldsystems.org/index.php/API_Public/sequence?ids={'|'.join(batch_ids)}&format=tsv"
    try:
        response = requests.get(url)
        response.raise_for_status()
        body = response.text

        lines = body.split('\r\n')
        for i in range(0, len(lines) - 1, 2): 
            info_line = lines[i]
            sequence_line = lines[i + 1]

            if info_line and sequence_line:
                bold_id = info_line.lstrip('>').split('|')[0]
                nucleotide = sequence_line.strip() 

                df.loc[df['processid'] == bold_id, 'nucleotide'] = nucleotide
    except Exception as e:
        print(f"Error fetching data for batch {index}: {e}")
        missing_bolds.extend(batch_ids)

    if index % 100 == 0:
        elapsed_time = time.time() - start_time
        print(f"Processed {index} batches, elapsed time: {elapsed_time:.2f} seconds")
    time.sleep(1)

threads = []
for i in range(0, len(df), batch_size):
    batch_ids = df['processid'][i:i + batch_size].tolist()
    thread = threading.Thread(target=fetch_data, args=(batch_ids, i // batch_size))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

df.to_csv('updated_species_samples.csv', index=False)

with open("missing_bolds.txt", "w") as f:
    for bold_id in missing_bolds:
        f.write(bold_id + "\n")

total_time = time.time() - start_time
print(f"Processing completed in {total_time:.2f} seconds")
    





Error fetching data for batch 11: 502 Server Error: Bad Gateway for url: http://v3.boldsystems.org/index.php/API_Public/sequence?ids=MNAA1040-17%7CBBLOE765-12%7CGMSVD4196-23%7CGMODL5212-22%7CGMODL8602-23%7CMNAL385-10%7CMNAD834-07%7CMNAL139-10%7CGMODL6583-22%7CGMODL5019-22%7CMNAO393-11%7CGMODL5232-22%7CMNAA910-07%7CGMODL8738-23%7CGMSVA3401-22%7CCOLMS5075-22%7CLPAB147-08%7CJBLRB1557-23%7CMNAJ1094-16%7CJBLRB1601-23%7CJBLRB1545-23%7CGMODL6559-22%7CGMODL6615-22%7CGMODL8764-23%7CMNAC1065-17%7CLOWCC465-05%7CMNAO309-11%7CLPAB136-08%7CCNWLG052-12%7CKSLEP1375-18%7CPOHL169-23%7CGMODL6631-22%7CGMSVD469-22%7CGMODL8715-23%7CLPAB131-08%7CLEPNG650-15%7CGMODL8774-23%7CLBCG2204-09%7CGMSCE434-20%7CMNAL381-10%7CGMODL5033-22%7CGMSVA5482-22%7CCNWBG2268-13%7CMNAJ998-09%7CBBYBM1403-20%7CHPPPJ727-13%7CLBCG2947-09%7CGMODL8610-23%7CLPAB124-08%7CLPAB614-08%7CGMSCK14663-23%7CLPALE3832-23%7CJBLRB1533-23%7CLPABB732-09%7CUSLEP384-10%7CGMODL8735-23%7CJBLRB1546-23%7CMNAL366-10%7CLPAB883-08%7CMNAP1151-13%7CJBLRB1590-23%


KeyboardInterrupt



In [2]:
df = pd.read_csv('updated_species_samples.csv')
df = df.dropna(subset = ['nucleotide'])

In [16]:
tform = transforms.Compose([transforms.Resize((64,64)),transforms.PILToTensor(),transforms.ConvertImageDtype(torch.float),transforms.Normalize(0.5,0.5)])
image_dataset = torchvision.datasets.ImageFolder("image_dataset/",transform=tform)

batch_size = 1000 

import random
import pickle

with open('genusname2genuslabel.pickle', 'rb') as handle:
    genusname2genuslabel = pickle.load(handle)

for i, (imgpath, specieslabel) in enumerate(image_dataset.imgs):
    imgpath = imgpath.replace("image_dataset/","")
    #print(imgpath)
    image_dataset.imgs[i] = (image_dataset.imgs[i][0],genusname2genuslabel[imgpath[:imgpath.index("/")]])

def new_unseen_get_imgs_bold_id(image_dataset,df):
  img2dna = dict()
  not_found_images = []
  for i, row in df.iterrows():
      url = row['image_urls'].split('|')[0]
      genus_name = row['genus_name']
      image_name_csv ='image_dataset/'+genus_name+'/'+url[url.rfind('/')+1:]
      
      trovato = False
      for img in image_dataset.imgs:
          if img[0] == image_name_csv:
              img2dna[img[0]]= row['processid']
              trovato = True
              break
      if not trovato:
          not_found_images.append(image_name_csv)
  return img2dna
img2dna = new_unseen_get_imgs_bold_id(image_dataset,df)

nucleotides = df[['nucleotide','species_name','genus_name','processid','image_urls']]

colonna_dna = df.loc[:,"nucleotide"]
#nucleotides.loc[:,'nucleotide'] = colonna_dna.apply(dataset_utils.one_hot_encoding)
nucleotides['string_nucleotides'] = nucleotides['nucleotide']
nucleotides.loc[:,'nucleotide'] = colonna_dna.apply(dataset_utils.one_hot_encoding)
random.seed(42)


/tmp/ipykernel_9139/2277970642.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nucleotides['string_nucleotides'] = nucleotides['nucleotide']


In [17]:
df.shape

(40354, 52)

In [18]:
img2dna_indices = dict()
for k,v in img2dna.items():
    #print(k)
    #print(v)
    dna_index = np.where(nucleotides['processid'].values == v)
    if dna_index[0].size > 0:
        #print(dna_index)
        dna_index = dna_index[0][0]
        
        for i,(name,_) in enumerate(image_dataset.imgs):
            if name == k:
                image_index = i
                break
        img2dna_indices[image_index] = dna_index
    else:
        raise Exception

In [19]:
len(img2dna_indices)

40050

In [20]:

all_not_expanded_one_hots = nucleotides['nucleotide'].to_numpy()
all_not_expanded_string_dnas= nucleotides['string_nucleotides'].to_numpy()
all_not_expanded_one_hots.shape

(40354,)

In [21]:
all_dnas = []
all_string_dnas = []
all_dna_labels = []
already_seen_dna_indices = set()
is_first_occurrence = []
not_missing = 0
for i in range(len(image_dataset.imgs)):
    if i not in img2dna_indices:
        print(f"skipping: image:{i} does not have a DNA")
        continue
    else:
        not_missing+=1
    all_dnas.append(torch.tensor(all_not_expanded_one_hots[img2dna_indices[i]]))
    all_string_dnas.append(all_not_expanded_string_dnas[img2dna_indices[i]])
    all_dna_labels.append(torch.tensor(image_dataset.imgs[i][1]))
    if img2dna_indices[i] not in already_seen_dna_indices:
        is_first_occurrence.append(True)
        already_seen_dna_indices.add(img2dna_indices[i])
    else:
        is_first_occurrence.append(False)
all_dnas = torch.stack(all_dnas)
all_dna_labels = torch.stack(all_dna_labels)
all_string_dnas = np.array(all_string_dnas)

skipping: image:6 does not have a DNA
skipping: image:43 does not have a DNA
skipping: image:667 does not have a DNA
skipping: image:698 does not have a DNA
skipping: image:1530 does not have a DNA
skipping: image:1531 does not have a DNA
skipping: image:1585 does not have a DNA
skipping: image:1586 does not have a DNA
skipping: image:1587 does not have a DNA
skipping: image:1588 does not have a DNA
skipping: image:1589 does not have a DNA
skipping: image:1590 does not have a DNA
skipping: image:1591 does not have a DNA
skipping: image:1592 does not have a DNA
skipping: image:1593 does not have a DNA
skipping: image:1594 does not have a DNA
skipping: image:1595 does not have a DNA
skipping: image:1596 does not have a DNA
skipping: image:1598 does not have a DNA
skipping: image:1599 does not have a DNA
skipping: image:1600 does not have a DNA
skipping: image:1601 does not have a DNA
skipping: image:1602 does not have a DNA
skipping: image:1603 does not have a DNA
skipping: image:1693 do

In [22]:
all_dnas.shape
not_missing

40050

In [23]:
len(image_dataset.imgs)

41258

In [25]:
from torch.utils.data import Dataset, DataLoader
batch_size = 1000
class WholeDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.targets = data.targets#torch.tensor(targets)
        #self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index][0]
        y = self.targets[index]
        
        return x, y
    
    def __len__(self):
        return len(self.data)
        
whole_dataset = WholeDataset(image_dataset)
n_classes = np.unique(whole_dataset.targets).shape[0]
whole_loader = torch.utils.data.DataLoader(whole_dataset, batch_size=1,shuffle=False, num_workers=0)


batch_images_list = []
batch_image_labels_list = []
with torch.no_grad():
    for i,(batch, targets) in enumerate(whole_loader):
        if i not in img2dna_indices:
            continue # we skip if an image doesnt have a DNA
        batch_images_list.append(batch.numpy())
        batch_image_labels_list.append(targets.numpy()) 
all_images= np.concatenate(batch_images_list)
all_labels= np.concatenate(batch_image_labels_list)

In [26]:
all_labels

array([  0,   0,   0, ..., 291, 291, 291])

In [27]:
all_dna_labels

tensor([235, 235, 235,  ..., 311, 311, 311])

In [28]:
boldids = dataset_utils.image_filenames_from_df(df)

In [43]:
genusname2genuslabel

{'Abax': 234,
 'Acleris': 235,
 'Acordulecera': 231,
 'Acrolophus': 232,
 'Acronicta': 233,
 'Adalia': 236,
 'Adelpha': 157,
 'Adrastus': 161,
 'Aedes': 160,
 'Agabus': 158,
 'Agonum': 159,
 'Agriophara': 175,
 'Agriotes': 174,
 'Agrotis': 37,
 'Agrypnus': 40,
 'Aleochara': 42,
 'Aloconota': 38,
 'Alosterna': 41,
 'Amara': 39,
 'Amaxia': 138,
 'Ametastegia': 140,
 'Amischa': 142,
 'Ammophila': 141,
 'Amorbia': 139,
 'Ampedus': 164,
 'Amphimallon': 167,
 'Anastrangalia': 168,
 'Andrena': 163,
 'Anisodactylus': 165,
 'Anisotoma': 166,
 'Anomis': 162,
 'Anopheles': 33,
 'Anoplodera': 35,
 'Antaeotricha': 30,
 'Anthicus': 31,
 'Anthidium': 34,
 'Anthobium': 32,
 'Aoplus': 36,
 'Apamea': 29,
 'Apion': 330,
 'Aplotarsus': 329,
 'Aproaerema': 331,
 'Araptus': 332,
 'Ardozyga': 327,
 'Arge': 328,
 'Argyrotaenia': 61,
 'Arhopalus': 60,
 'Armigeres': 59,
 'Astraptes': 62,
 'Athalia': 191,
 'Atheta': 188,
 'Athous': 189,
 'Atomaria': 190,
 'Aulacophora': 192,
 'Automeris': 187,
 'Bactrocera': 299

In [46]:
all_labels

array([  0,   0,   0, ..., 291, 291, 291])

In [49]:
all_dna_labels

tensor([235, 235, 235,  ..., 311, 311, 311])

In [30]:
import scipy.io as io
all_dataset = io.loadmat('unseen_insect_dataset.mat')

In [50]:
#all_dataset = dict()
#all_dataset['all_images'] = all_images 
#all_dataset['all_dnas'] = all_dnas.numpy()
#all_dataset['all_string_dnas'] = all_string_dnas
all_dataset['all_genus_labels'] = (all_dna_labels+1)
#all_dataset['all_boldids']= np.array(boldids)
savemat('unseen_insect_dataset.mat',all_dataset)

In [54]:
all_dataset['all_genus_labels']

array([[236, 236, 236, ..., 312, 312, 312]])

In [13]:
vero_dna = 'ATTATATTTTATTTTTGGTATTTGATCTGGAATAGTAGGAACTTCTTTAAGTTTATTAATTCGTGCCGAATTAGGAAACCCTGGCTCATTAATTGGAGATGATCAAATCTATAATACAATTGTAACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATCATAATTGGAGGATTTGGAAATTGATTAGTTCCTTTAATGTTAGGTGCCCCAGATATAGCATTTCCACGAATAAATAATATAAGTTTTTGATTACTCCCCCCATCTTTAACTCTTTTAGTCTCAAGAAGAATTGTTGAAAATGGAGCAGGAACAGGATGAACCGTTTACCCCCCTTTATCTTCTAATATTGCTCACGGAGGAGGATCTGTAGATTTAGCAATTTTTTCCCTTCACTTAGCTGGAATTTCTTCAATTTTAGGAGCTGTAAATTTTATTACAACAGTAATTAATATACGAGCTAATGGAATATCTTTTGATCGAATACCTTTATTTGTTTGAGCTGTTGCAATTACAGCCCTTTTATTACTTCTTTCTCTTCCTGTTCTTGCAGGAGCAATTACAATATTATTAACTGATCGAAACTTAAATACTTCCTTCTTTGATCCAGCTGGAGGAGGAGATCCTATTCTTTATCAACATTTA'

In [14]:
df.loc[100]['nucleotide']

'ATTATATTTTATTTTTGGTATTTGATCTGGAATAGTAGGAACTTCTTTAAGTTTATTAATTCGTGCCGAATTAGGAAACCCTGGCTCATTAATTGGAGATGATCAAATCTATAATACAATTGTAACTGCTCATGCTTTTATTATAATTTTTTTTATAGTTATACCTATCATAATTGGAGGATTTGGAAATTGATTAGTTCCTTTAATGTTAGGTGCCCCAGATATAGCATTTCCACGAATAAATAATATAAGTTTTTGATTACTCCCCCCATCTTTAACTCTTTTAGTCTCAAGAAGAATTGTTGAAAATGGAGCAGGAACAGGATGAACCGTTTACCCCCCTTTATCTTCTAATATTGCTCACGGAGGAGGATCTGTAGATTTAGCAATTTTTTCCCTTCACTTAGCTGGAATTTCTTCAATTTTAGGAGCTGTAAATTTTATTACAACAGTAATTAATATACGAGCTAATGGAATATCTTTTGATCGAATACCTTTATTTGTTTGAGCTGTTGCAATTACAGCCCTTTTATTACTTCTTTCTCTTCCTGTTCTTGCAGGAGCAATTACAATATTATTAACTGATCGAAACTTAAATACTTCCTTCTTTGATCCAGCTGGAGGAGGAGATCCTATTCTTTATCAACATTTA'

In [15]:
vero_dna == df.loc[100]['nucleotide']

True

In [ ]:
url = f"http://v3.boldsystems.org/index.php/API_Public/sequence?ids=PLLBQ869-20&format=tsv"


In [34]:
all_string_dnas[0]

'AACACTATATTTCATTTTTGGTATTTGAGCAGGTATAGTGGGAACCTCTTTAAGACTATTAATTCGAGCAGAATTAGGAAACCCGGGATCATTAATTGGAGATGATCAAATTTATAATACTATTGTAACAGCTCATGCATTTATTATAATTTTTTTTATAGTAATACCTATCATAATTGGTGGATTTGGTAATTGATTAGTACCACTAATATTAGGAGCCCCTGATATAGCTTTTCCACGAATAAATAATATAAGTTTTTGATTATTACCACCTTCTATTATACTTTTAATTTCAAGTAGAATTGTAGAAAATGGAGCAGGAACAGGATGAACGGTTTACCCCCCCCTTTCATCTAATATCGCTCATAGTGGTAGTTCTGTCGATTTAGCTATTTTTTCTTTACATTTAGCTGGTATTTCATCAATTTTAGGAGCAGTAAATTTTATTACAACAATCATTAATATACGTCCTAATAATATATCACTAGATCAAATACCTTTATTTGTATGAGCTGTTGGAATTACAGCTTTATTATTACTTTTATCTTTACCAGTATTAGCTGGAGCTATTACTATATTATTAACAGATCGGAATTTAAATACATCATTTTTTGATCCTGCTGGAGGAGGAGATCCAATTTTATATCAACATTTATTT'

In [49]:
(dataset_utils.one_hot_encoding(all_string_dnas[0])==all_dnas[0].numpy()).all()

True

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [39]:
df.loc[df['nucleotide']=='AACACTATATTTCATTTTTGGTATTTGAGCAGGTATAGTGGGAACCTCTTTAAGACTATTAATTCGAGCAGAATTAGGAAACCCGGGATCATTAATTGGAGATGATCAAATTTATAATACTATTGTAACAGCTCATGCATTTATTATAATTTTTTTTATAGTAATACCTATCATAATTGGTGGATTTGGTAATTGATTAGTACCACTAATATTAGGAGCCCCTGATATAGCTTTTCCACGAATAAATAATATAAGTTTTTGATTATTACCACCTTCTATTATACTTTTAATTTCAAGTAGAATTGTAGAAAATGGAGCAGGAACAGGATGAACGGTTTACCCCCCCCTTTCATCTAATATCGCTCATAGTGGTAGTTCTGTCGATTTAGCTATTTTTTCTTTACATTTAGCTGGTATTTCATCAATTTTAGGAGCAGTAAATTTTATTACAACAATCATTAATATACGTCCTAATAATATATCACTAGATCAAATACCTTTATTTGTATGAGCTGTTGGAATTACAGCTTTATTATTACTTTTATCTTTACCAGTATTAGCTGGAGCTATTACTATATTATTAACAGATCGGAATTTAAATACATCATTTTTTGATCCTGCTGGAGGAGGAGATCCAATTTTATATCAACATTTATTT']['genus_name']

16    Acleris
38    Acleris
Name: genus_name, dtype: object

In [40]:
genusname2genuslabel['Acleris']

235

In [35]:
df['nucleotide']

0        AACGTTATATTTCATTTTTGGTATTTGAGCAGGTATAGTTGGAACT...
1        ATTATATTTCATTTTTGGTATTTGAGCAGGTATAGTAGGAACTTCT...
2        ----------------------------------------------...
3        AACATTATATTTTATTTTTGGTATTTGAGCAGGTATAGTAGGAACT...
4        AACCTTATATTTTATTTTTGGAATTTGAGCAGGTATAGTGGGAACT...
                               ...                        
41567    ATTATATTTTATTTTTGGAGCTTGAGCCGGAATAGTAGGAACATCC...
41568    AATAGTAGGAACATCATTAAGAATTCTAATTCGAGCTGAATTAGGA...
41569    ATTATATTTTATTTTTGGAGCTTGAGCCGGAATAGTAGGAACATCC...
41570    TTTATATTTTATTTTTGGAGCATGAGCCGGAATAGTAGGAACATCA...
41571    TTTATATTTTATTTTTGGAGCGTGAGCCGGAATAGTAGGAACATCA...
Name: nucleotide, Length: 40354, dtype: object

(2, 41233)